In [53]:
import copy
import time
import numpy as np
import open3d as o3
from probreg import cpd, filterreg, gmmtree, l2dist_regs, bcpd

In [54]:
use_cuda = True
if use_cuda:
    import cupy as cp
    to_cpu = cp.asnumpy
    cp.cuda.set_allocator(cp.cuda.MemoryPool().malloc)
else:
    cp = np
    to_cpu = lambda x: x

In [55]:
def vis(source, target, result):
    source.paint_uniform_color([1, 0, 0]) #r
    target.paint_uniform_color([0, 1, 0]) #g
    result.paint_uniform_color([0, 0, 1]) #b
    o3.visualization.draw_geometries([source, target, result])

In [56]:
left = o3.io.read_point_cloud("input/left.xyz", format='xyz')
full = o3.io.read_point_cloud("input/full.xyz", format='xyz')
right = o3.io.read_point_cloud("input/right.xyz", format='xyz')

ROOM = o3.io.read_point_cloud("input/ROOM.xyz", format='xyz')
MAP = o3.io.read_point_cloud("input/MAP.xyz", format='xyz')

In [57]:
left_01 = left.voxel_down_sample(voxel_size=0.01)
right_01 = right.voxel_down_sample(voxel_size=0.01)
MAP_01 = MAP.voxel_down_sample(voxel_size=0.01)
ROOM_01 = ROOM.voxel_down_sample(voxel_size=0.01)
full_01 = full.voxel_down_sample(voxel_size=0.01)

left_005 = left.voxel_down_sample(voxel_size=0.005)
full_005 = full.voxel_down_sample(voxel_size=0.005)

## CPD

In [62]:
%%time
tf_param, _, _ = cpd.registration_cpd(left_01, full_01)


CPU times: user 46.2 s, sys: 35.9 s, total: 1min 22s
Wall time: 35.7 s


In [63]:
result_cpd_01 = copy.deepcopy(left_01)
result_cpd_01.points = tf_param.transform(result_cpd_01.points)
vis(left_01, full_01, result_cpd_01)

In [64]:
%%time
tf_param, _, _ = cpd.registration_cpd(left_01, right_01)

CPU times: user 18.9 s, sys: 30.3 s, total: 49.2 s
Wall time: 9.07 s


In [65]:
result_cpd_01 = copy.deepcopy(left_01)
result_cpd_01.points = tf_param.transform(result_cpd_01.points)
vis(left_01, right_01, result_cpd_01)


## FilterReg

0.005:

In [48]:
%%time
tf_param, _, _ = filterreg.registration_filterreg(left_005, full_005)

CPU times: user 939 ms, sys: 18.7 ms, total: 957 ms
Wall time: 204 ms


In [49]:
result_fr_005 = copy.deepcopy(left_005)
result_fr_005.points = tf_param.transform(result_fr_005.points)
vis(left_005, full_005, result_fr_005)

0.01:

In [60]:
%%time
tf_param, _, _ = filterreg.registration_filterreg(left_01, full_01)

CPU times: user 859 ms, sys: 45.5 ms, total: 905 ms
Wall time: 189 ms


In [61]:
result_fr_01 = copy.deepcopy(left_01)
result_fr_01.points = tf_param.transform(result_fr_01.points)
vis(left_01, full_01, result_fr_01)

left -> right

In [35]:
%%time
tf_param, _, _ = filterreg.registration_filterreg(left_01, right_01)


CPU times: user 708 ms, sys: 101 ms, total: 809 ms
Wall time: 141 ms


In [36]:
result_fr_01_lr = copy.deepcopy(left_01)
result_fr_01_lr.points = tf_param.transform(result_fr_01_lr.points)

vis(left_01, right_01, result_fr_01_lr)

ROOM -> MAP

In [86]:
start_time = time.time()
tf_param, _, _ = filterreg.registration_filterreg(ROOM_01, MAP_01)
print(time.time() - start_time)

result_fr_01_rm = copy.deepcopy(ROOM_01)
result_fr_01_rm.points = tf_param.transform(result_fr_01_rm.points)

MemoryError: std::bad_alloc

In [ ]:
vis(ROOM_01, MAP_01, result_fr_01_rm)

## GMM Tree

In [37]:
%%time
tf_param, _ = gmmtree.registration_gmmtree(left_01, full_01)

CPU times: user 1min 38s, sys: 280 ms, total: 1min 39s
Wall time: 8.61 s


In [38]:
result_gmm_01 = copy.deepcopy(left_01)
result_gmm_01.points = tf_param.transform(result_gmm_01.points)
vis(left_01, full_01, result_gmm_01)

left->right

In [39]:
%%time
tf_param, _ = gmmtree.registration_gmmtree(left_01, right_01)

CPU times: user 57.5 s, sys: 391 ms, total: 57.9 s
Wall time: 5.42 s


In [40]:
result_gmm_01 = copy.deepcopy(left_01)
result_gmm_01.points = tf_param.transform(result_gmm_01.points)
vis(left_01, right_01, result_gmm_01)

## SVR

In [41]:
%%time
tf_param = l2dist_regs.registration_svr(left_01, full_01)

CPU times: user 4.41 s, sys: 21.3 ms, total: 4.43 s
Wall time: 844 ms


In [42]:
result_svr_01 = copy.deepcopy(left_01)
result_svr_01.points = tf_param.transform(result_svr_01.points)

vis(left_01, full_01, result_svr_01)

left->right

In [43]:
%%time
tf_param = l2dist_regs.registration_svr(left_01, right_01)

CPU times: user 1.95 s, sys: 0 ns, total: 1.95 s
Wall time: 237 ms


In [44]:
result_svr_01 = copy.deepcopy(left_01)
result_svr_01.points = tf_param.transform(result_svr_01.points)

vis(left_01, right_01, result_svr_01)

# BCPD

left->full

In [51]:
%%time
tf_param = bcpd.registration_bcpd(left_01, full_01)

CPU times: user 3min 1s, sys: 1min 24s, total: 4min 25s
Wall time: 1min 43s


In [52]:
result_bcpd_01 = copy.deepcopy(left_01)
result_bcpd_01.points = tf_param.transform(result_svr_01.points)

vis(left_01, right_01, result_bcpd_01)